In [1]:
import time
import numpy as np
import os

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance


import gensim.downloader as api
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.matutils import softcossim
from gensim.models import WordEmbeddingSimilarityIndex
from gensim.similarities import SoftCosineSimilarity, SparseTermSimilarityMatrix
from gensim.models import FastText 

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ravigv\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import spacy
import warnings
warnings.filterwarnings("ignore") 

In [3]:
%%time
# Setting path 
# Get working directory
import os
print(os.getcwd())

# Set working directory

os.chdir('C:\\Users\\ravigv\\Documents\\cosine_sim\\') 
print(os.getcwd())

C:\Users\ravigv
C:\Users\ravigv\Documents\cosine_sim
Wall time: 965 µs


In [7]:
%%time
query = "apple apple is fruit"
doc = "apple is phone"

Wall time: 0 ns


In [5]:
%%time
# Cosine similarity countvectorizer - BOW model

def cosine_distance_countvectorizer_method(s1, s2):
    
    # sentences to list
    allsentences = [s1 , s2]
    
    # packages
    from sklearn.feature_extraction.text import CountVectorizer
    from scipy.spatial import distance
    
    # text to vector
    vectorizer = CountVectorizer()
    all_sentences_to_vector = vectorizer.fit_transform(allsentences)
    text_to_vector_v1 = all_sentences_to_vector.toarray()[0].tolist()
    text_to_vector_v2 = all_sentences_to_vector.toarray()[1].tolist()
    
    # distance of similarity
    cosine = distance.cosine(text_to_vector_v1, text_to_vector_v2)
    print('Similarity of two sentences are equal to ',round((1-cosine)*100,2),'%')
    return cosine


# Calling the function
cosine_distance_countvectorizer_method(query, doc)




Similarity of two sentences are equal to  70.71 %
Wall time: 30.9 ms


0.2928932188134524

In [8]:
%%time
# Cosine similarity tfidfvectorizer method

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def get_tf_idf_query_OneDoc_similarity(docs, query):
    docTFIDF = TfidfVectorizer().fit_transform([docs])
    queryTFIDF = TfidfVectorizer().fit([docs])
    queryTFIDF = queryTFIDF.transform([query])

    cosineSimilarities = cosine_similarity(queryTFIDF, docTFIDF).flatten()
    print('Similarity of two sentences are equal to ',cosineSimilarities)
    return cosineSimilarities

get_tf_idf_query_OneDoc_similarity(doc, query)


Similarity of two sentences are equal to  [0.77459667]
Wall time: 106 ms


array([0.77459667])

In [7]:
%%time
#  linear kernel cosine similarity tfidfvectorizer method

def lk_cosim_tfidfvectorizer_method2(docs, query):
    docTFIDF = TfidfVectorizer().fit_transform([docs])
    queryTFIDF = TfidfVectorizer().fit([docs])
    queryTFIDF = queryTFIDF.transform([query])

    lk_cosim = linear_kernel(queryTFIDF, docTFIDF).flatten()
    print('Similarity of two sentences are equal to ',lk_cosim)
    return lk_cosim


# Compute cosine similarity matrix
lk_cosim_tfidfvectorizer_method2(doc, query)



Similarity of two sentences are equal to  [0.81649658]
Wall time: 0 ns


array([0.81649658])

In [8]:
%%time
# Cosine similarity using spacy small model

def spacy_cosim(query, doc):
    nlp = spacy.load("en_core_web_sm")
    doc1 = nlp(query)
    doc2 = nlp(doc)
    sp_cosim = doc1.similarity(doc2)
    print('Similarity of two sentences are equal to ',sp_cosim)
    return sp_cosim


# Calling the function
spacy_cosim(query, doc)

Similarity of two sentences are equal to  0.8865374078515251
Wall time: 4.58 s


0.8865374078515251

In [9]:
%%time
# Cosine similarity using spacy medium model
import spacy

def spacy_cosim(query, doc):
    nlp = spacy.load("en_core_web_md")
    doc1 = nlp(query)
    doc2 = nlp(doc)
    sp_cosim = doc1.similarity(doc2)
    print('Similarity of two sentences are equal to ',sp_cosim)
    return sp_cosim


# Calling the function
spacy_cosim(query, doc)

Similarity of two sentences are equal to  0.7795348503474273
Wall time: 5.01 s


0.7795348503474273

In [10]:
%%time


# Spacy cosine similarity with large model 
def spacy_cosim(query, doc):
    nlp = spacy.load("en_core_web_lg")
    doc1 = nlp(query)
    doc2 = nlp(doc)
    sp_cosim = doc1.similarity(doc2)
    print('Similarity of two sentences are equal to ',sp_cosim)
    return sp_cosim


# Calling the function
spacy_cosim(query, doc)


Similarity of two sentences are equal to  0.7795348503474273
Wall time: 12.1 s


0.7795348503474273

In [12]:
%%time
# SoftCosine similarity gensim fasttext model

def gensim_cosim(query, doc):
    
    query1 = query.split()
    doc1 = doc.split()
    
    # Download the FastText model
    fasttext_model300 = api.load('fasttext-wiki-news-subwords-300')
    # sentences to list
    docs = [query1, doc1]
    # Prepare a dictionary and a corpus.
    dictionary = corpora.Dictionary(docs)
    # Prepare the similarity matrix
    similarity_index = WordEmbeddingSimilarityIndex(fasttext_model300)
    similarity_matrix = SparseTermSimilarityMatrix(similarity_index, dictionary)

    # Convert the sentences into bag-of-words vectors.
    bow_query = dictionary.doc2bow(query1)
    bow_doc = dictionary.doc2bow(doc1)

# Compute soft cosine similarity
    print(similarity_matrix.inner_product(bow_query, bow_doc, normalized=True))
    return similarity_matrix.inner_product(bow_query, bow_doc, normalized=True)

    
    
gensim_cosim(query, doc)

0.7146543
Wall time: 5min 8s


0.7146543

In [16]:
%%time

# Custom cosine similarity 

import math
import re
from collections import Counter

WORD = re.compile(r"\w+")


def get_cosine(vec1, vec2):
    intersection = set(vec1.keys()) & set(vec2.keys())
    numerator = sum([vec1[x] * vec2[x] for x in intersection])

    sum1 = sum([vec1[x] ** 2 for x in list(vec1.keys())])
    sum2 = sum([vec2[x] ** 2 for x in list(vec2.keys())])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
   
    if not denominator:
        return  0.0
    else:
        return float(numerator) / denominator


def text_to_vector(text):
    words = WORD.findall(text)
    return Counter(words)


text1 = "apple is fruit"
text2 = "apple is phone"

vector1 = text_to_vector(text1)
vector2 = text_to_vector(text2)

cosine = get_cosine(vector1, vector2)

print("Vector1:", vector1)
print("Vector2:", vector2)
print("Cosine:", cosine)

Vector1: Counter({'apple': 1, 'is': 1, 'fruit': 1})
Vector2: Counter({'apple': 1, 'is': 1, 'phone': 1})
Cosine: 0.6666666666666667
Wall time: 0 ns
